In [1]:
import tabula
import pandas as pd
import numpy as np
import requests
import json
import time


## Get Yelp Matches from MOPD Data

In [2]:
pdf_filename = 'Restaurant_Compliance_July_2019_HM.pdf'
tab = tabula.read_pdf(pdf_filename,encoding='utf-8',pages='1-12')
cols = ['id','business','fulladdress','compliant']
df = pd.DataFrame(columns=cols)


In [3]:
for i in range(len(tab)-1):
    orig_page = tab[i]
    page = orig_page[(orig_page['Address'].notnull())].copy()
    page['compliant'] = [1 if x =='X' else 0 for x in page['Compliance']]
    page['Sl'] = page['Sl'].astype(int)
    df_filtered = page[['Sl','Business','Address','compliant']]
    df_filtered.columns=cols
    df = df.append(df_filtered)
    

In [4]:
string = '233 Broadway, New York, NY 10279'
address = string.split(',')[0]
city = string.split(',')[1].strip(' ')
state  = string.split(',')[2].split(' ')[1].strip(' ')
zipcode = string.split(',')[2].split(' ')[2].strip(' ')

print(address)
print(city)
print(state)
print(zipcode)

233 Broadway
New York
NY
10279


In [5]:
df['address'] = df['fulladdress'].str.split(',',expand=True)[0]
df['city'] = df['fulladdress'].str.split(',',expand=True)[1].str.strip(' ')
df['statezip'] = df['fulladdress'].str.split(',',expand=True)[2]
df['zip'] = df['statezip'].str.extract(r'(\d\d\d\d\d)')
df['state'] = 'NY'
df.head()


,id,business,fulladdress,compliant,address,city,statezip,zip,state
1,1,Just Salad,"233 Broadway, New York, NY 10279",1,233 Broadway,New York,NY 10279,10279,NY
2,2,Fowler and Wells,"5 Beekman St, New York, NY 10038",0,5 Beekman St,New York,NY 10038,10038,NY
3,3,Birch Coffee,"8 Spruce St, New York, NY 10038",1,8 Spruce St,New York,NY 10038,10038,NY
4,4,"Wxyz Bar,","49 Ann St, New York, NY 10038",1,49 Ann St,New York,NY 10038,10038,NY
5,5,Poke Bowl,"104 Fulton St, New York, NY10038",1,104 Fulton St,New York,NY10038,10038,NY


In [7]:
yelp_df = pd.DataFrame(columns=['id','name','latitude','longitude','address1','address2','address3','city','state','zip_code','phone','compliant'])
api_key='PZRYQEGHjvw5dEh0RKu0XmQhAzwNXsAY7woyCv3dC0LXTh2kxLEY_QI4dM4Oj4N_F94_5Liesc8SoA51HLYlTnYFHPFisD0Akkr4IwzieUcXy8OR-1sEl6WRVxiTX3Yx'
headers = {'Authorization': 'Bearer %s' % api_key}
url = 'https://api.yelp.com/v3/businesses/matches'

counter = 0
for i in range(len(df)):
    counter = counter+1
    compliant = df['compliant'].values[i]
    if counter % 100 == 0:
        print('Working on request %s...' % (counter))
    mopd_name = df['business'].values[i]
    mopd_address = df['address'].values[i]
    mopd_zip = df['zip'].values[i]
    mopd_city = df['city'].values[i]
    mopd_state = df['state'].values[i]
    
    params = {'name':mopd_name,'address1':mopd_address,'zip_code':mopd_zip,
              'city':mopd_city,'state':mopd_state,'country':'US'}
    req=requests.get(url, params=params, headers=headers)
    parsed = json.loads(req.text)
    try:
        business = parsed['businesses'][0]
        yelp_id = business['id']
        yelp_name = business['name']
        latitude = business['coordinates']['latitude']
        longitude = business['coordinates']['longitude']
        address1 = business['location']['address1']
        address2 = business['location']['address2']
        address3 = business['location']['address3']
        yelp_city = business['location']['city']
        yelp_state = business['location']['state']
        yelp_zip = business['location']['zip_code']
        phone = business['phone']
        row = [yelp_id,yelp_name,latitude,longitude,address1,address2,address3,
               yelp_city,yelp_state,yelp_zip,phone,compliant]
        yelp_df.loc[len(yelp_df)] = row
    except:
        row = [np.nan,mopd_name,np.nan,np.nan,mopd_address,np.nan,np.nan,
              mopd_city,mopd_state,mopd_zip,np.nan,compliant]
        yelp_df.loc[len(yelp_df)] = row

Working on request 100...
Working on request 200...
Working on request 300...
Working on request 400...
Working on request 500...


In [8]:
print('Number of records: %s' % (len(yelp_df)))
print('Number of Yelp matches: %s' % (len(yelp_df[(yelp_df['id'].notnull())])))

# Limit dataset to Yelp matches
yelp_mopd_matches = yelp_df[(yelp_df['id'].notnull())]


Number of records: 524
Number of Yelp matches: 188


In [9]:
yelp_mopd_matches.head()


,id,name,latitude,longitude,address1,address2,address3,city,state,zip_code,phone,compliant
0,14BLy22t6EKuPO5aAQ6R8g,Just Salad,40.712709,-74.008401,233 Broadway,,,New York,NY,10279,+12122441111,1
2,ejZjFbR_Z3He5pi699eJKg,Birch Coffee,40.710840,-74.005672,8 Spruce St,None,,New York,NY,10038,+12126861444,1
3,K7c1bkgmmdiTcsQ5afGFNQ,W XYZ Bar,40.710350,-74.006790,49-53 Ann St,None,None,New York,NY,10038,+12125130003,1
4,t4B9GkAuiYIjFCt9SlK3EQ,Poke Bowl,40.709569,-74.006834,104 Fulton St,,None,New York,NY,10038,+16463719773,1
6,j7NsPKpOQsIHBbBYrBGZQg,Dunkin',40.708140,-74.006860,80 John St,,,New York,NY,10038,+12122320020,1


In [11]:
id_df = pd.DataFrame(columns=['id','name','image_url','url','review_count','rating','price'])

api_key='PZRYQEGHjvw5dEh0RKu0XmQhAzwNXsAY7woyCv3dC0LXTh2kxLEY_QI4dM4Oj4N_F94_5Liesc8SoA51HLYlTnYFHPFisD0Akkr4IwzieUcXy8OR-1sEl6WRVxiTX3Yx'
headers = {'Authorization': 'Bearer %s' % api_key}
url='https://api.yelp.com/v3/businesses/'

id_urls = [url+str(x) for x in yelp_mopd_matches['id'].tolist()]
counter = 0
for i in id_urls:
    counter = counter+1
    if counter % 20 == 0:
        print(i)
    req = requests.get(i,headers=headers)
    parsed = json.loads(req.text)
    try:
        yelp_id = parsed['id']
        name = parsed['name']
        image_url = parsed['image_url']
        url = parsed['url']
        review_count = parsed['review_count']
        rating = parsed['rating']
        price = parsed['price']
        row = [yelp_id,name,image_url,url,review_count,rating,price]
        id_df.loc[len(id_df)] = row
        print('Done!')
        time.sleep(2)
    except:
        time.sleep(2)

Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
https://api.yelp.com/v3/businesses/ziPLg64qxWsYiBBmCMbXPw
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
https://api.yelp.com/v3/businesses/MoiZlLUndZjTTXAFJXTHXA
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
https://api.yelp.com/v3/businesses/sCC7-hSdCkNPExejZT9BAQ
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
https://api.yelp.com/v3/businesses/xB5TkX3khfpLMxFi8Z1MDw
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
https://api.yelp.com/v3/businesses/nnNMrTTLohVjhKDXAlXUJA
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
https://api.yelp.com

In [13]:
print(id_df.columns)
print(yelp_mopd_matches.columns)
print(df.columns)
all_matches = id_df.merge(yelp_mopd_matches,
                         on=['id','name'],
                         how='left',
                         indicator=True)

all_matches['_merge'].value_counts()
all_matches.drop(['_merge'],axis=1,inplace=True)
all_matches.to_csv('Yelp-MOPD-10-26-20.csv',index=False)

Index(['id', 'name', 'image_url', 'url', 'review_count', 'rating', 'price'], dtype='object')
Index(['id', 'name', 'latitude', 'longitude', 'address1', 'address2',
       'address3', 'city', 'state', 'zip_code', 'phone', 'compliant'],
      dtype='object')
Index(['id', 'business', 'fulladdress', 'compliant', 'address', 'city',
       'statezip', 'zip', 'state'],
      dtype='object')


## Get Restaurants from Yelp Search

In [154]:
yelp_search_df = pd.DataFrame(columns=['id','name','image_url','url','review_count','rating',
                                       'price','address1','address2','address3','city','state',
                                       'zip_code','phone'])

yelp_search_df.to_csv('Yelp_Restaurants.csv',index=False)


In [150]:
nyc_zips_df = pd.read_csv('https://raw.githubusercontent.com/erikgregorywebb/nyc-housing/master/Data/nyc-zip-codes.csv')
nyc_zips = [str(x) for x in nyc_zips_df['ZipCode'].tolist()]
nyc_zips[0:5]

['10453', '10457', '10460', '10458', '10467']

In [155]:
def get_restaurants_by_zip(start, stop, step, zip_code):
    api_key='PZRYQEGHjvw5dEh0RKu0XmQhAzwNXsAY7woyCv3dC0LXTh2kxLEY_QI4dM4Oj4N_F94_5Liesc8SoA51HLYlTnYFHPFisD0Akkr4IwzieUcXy8OR-1sEl6WRVxiTX3Yx'
    headers = {'Authorization': 'Bearer %s' % api_key}
    url='https://api.yelp.com/v3/businesses/search'
    for i in range(start, stop, step):
#        print(i)
        params = {'location':z,'categories':'restaurants, All','limit':'50','offset':i}
        # Making a get request to the API
        req=requests.get(url, params=params, headers=headers)

        # printing the text from the response 
        parsed = json.loads(req.text)
        businesses = parsed['businesses']
        for b in businesses:
            try:
                yelp_id = b['id']
                name = b['name']
                image = b['image_url']
                url = b['url'] = url
                review_count = b['review_count']
                rating = b['rating']
                price = b['price']
                address1 = b['location']['address1']
                address2 = b['location']['address2']
                address3 = b['location']['address3']
                city = b['location']['city']
                state = b['location']['state']
                nyczip = b['location']['zip_code']
                phone = b['phone']
                row = [yelp_id,name,image,url,review_count,rating,price,address1,address2,address3,city,state,nyczip,phone]
                yelp_search_df.loc[len(yelp_search_df)] = row
            except:
                pass
        yelp_search_df.to_csv('Yelp_Restaurants.csv', mode='a', header=False)

In [182]:
api_key='PZRYQEGHjvw5dEh0RKu0XmQhAzwNXsAY7woyCv3dC0LXTh2kxLEY_QI4dM4Oj4N_F94_5Liesc8SoA51HLYlTnYFHPFisD0Akkr4IwzieUcXy8OR-1sEl6WRVxiTX3Yx'
headers = {'Authorization': 'Bearer %s' % api_key}
url='https://api.yelp.com/v3/businesses/search'
start = 0
stop = 100
step = 50
for i in range(start, stop, step):
    params = {'location':'10026','categories':'restaurants, All','limit':'50','offset':i}
    # Making a get request to the API
    req=requests.get(url, params=params, headers=headers)

    # printing the text from the response 
    parsed = json.loads(req.text)
    businesses = parsed['businesses']

In [184]:
get_restaurants_by_zip(0,1000,50,'10026')

In [20]:
for z in nyc_zips:
    print('Working on %s...' % (z))
    get_restaurants_by_zip(0,1000,50,z)
    

In [159]:
# eliminate duplicates
print(len(yelp_search_df))
yelp_search_df.drop_duplicates(subset='id',inplace=True)
print(len(yelp_search_df))
yelp_search_df['id'].value_counts()

8753
8748


UH_aksIxzGPPyhWZJbTP7A    1
YllbMI89Czj6rrN-XMdUNw    1
R7pAORlR0pslk_uJ8FWa0g    1
7UJRw4Klt5yfE7q9vznEHA    1
ZLo3HRSSvN--Mw4eCBLxCg    1
                         ..
HV8xairb2qct7wSHqNwqXw    1
zRwnU8qW83e0RGieeBkfnw    1
hXzbLyw9J093OozTT8GYwQ    1
p7HWTeR3hhMOasQLcNZBLg    1
UO4kyXfdQIlAhM-TRgM9cw    1
Name: id, Length: 8748, dtype: int64

In [160]:
yelp_search_df.head()



,id,name,image_url,url,review_count,rating,price,address1,address2,address3,city,state,zip_code,phone
0,b6jOwyX4iaagw8YjXqq1sA,Antonio's Trattoria,https://s3-media3.fl.yelpcdn.com/bphoto/VnG_ke...,https://api.yelp.com/v3/businesses/search,806,4.5,$$,2370 Belmont Ave,None,,Bronx,NY,10458,+17187336630
1,XzUW9MvEGOFs9_aNj11j2Q,Dinosaur Bar-B-Que,https://s3-media1.fl.yelpcdn.com/bphoto/YWMehM...,https://api.yelp.com/v3/businesses/search,3077,4.0,$$,700 W 125th St,,,Harlem,NY,10027,+12126941777
2,Dwg-RgHUXsBfAi7G0V8LIg,Green Juice Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/dHF2qE...,https://api.yelp.com/v3/businesses/search,201,5.0,$,4316 Broadway,None,,New York,NY,10033,+16469187730
3,tLZfepIfSf0a80zHQTbTGQ,Trattoria Zero Otto Nove,https://s3-media2.fl.yelpcdn.com/bphoto/lvgheh...,https://api.yelp.com/v3/businesses/search,772,4.0,$$,2357 Arthur Ave,,,Bronx,NY,10458,+17182201027
4,HVeQNs_7zh3m-xCZ1ugslA,Beans and Vines,https://s3-media4.fl.yelpcdn.com/bphoto/8Adkld...,https://api.yelp.com/v3/businesses/search,421,4.0,$$,4799 Broadway,,,Inwood,NY,10034,+16469280209


In [167]:
# match with MOPD data

final_df = pd.concat([yelp_search_df,id_df])
print(len(yelp_search_df),len(id_df),len(final_df))
final_df.drop_duplicates(subset='id',inplace=True)
final_df['id'].value_counts().head()
print(len(final_df))
final_df.to_csv('Yelp_Restaurants_All.csv',index=False)

8748 183 8931
8875


In [170]:
final_df['name'].value_counts().head(50)


McDonald's                           107
Subway                                77
Popeyes Louisiana Kitchen             67
Domino's Pizza                        48
Burger King                           46
Papa John's Pizza                     36
Chipotle Mexican Grill                30
Crown Fried Chicken                   30
KFC                                   25
Wendy's                               21
Kennedy Fried Chicken                 18
White Castle                          17
IHOP                                  16
Golden Krust Caribbean Restaurant     15
Taco Bell                             15
Applebee's Grill + Bar                15
Checkers                              13
Texas Chicken & Burgers               11
Jimbo's Hamburger Palace              11
Shake Shack                            9
Five Guys                              9
Little Caesars                         8
Boston Market                          8
Caridad Restaurant                     8
Pizza Hut       

In [171]:
final_df[(final_df['name']=='Sweet Chick')]

,id,name,image_url,url,review_count,rating,price,address1,address2,address3,city,state,zip_code,phone
10778,iZeAlmKKvxnPu5oUKN-B3Q,Sweet Chick,https://s3-media3.fl.yelpcdn.com/bphoto/i6huGz...,https://api.yelp.com/v3/businesses/search,651,4.0,$$,341 Flatbush Ave,,None,Brooklyn,NY,11217,+17184847724
23678,4KHjnRNvrkKHchEznhlnSA,Sweet Chick,https://s3-media3.fl.yelpcdn.com/bphoto/-B7HnM...,https://api.yelp.com/v3/businesses/search,2018,4.0,$$,164 Bedford Ave,,,Brooklyn,NY,11211,+13477254793
23794,8F81LrdFm6P5kkFpsw0n0w,Sweet Chick,https://s3-media3.fl.yelpcdn.com/bphoto/_KVRRk...,https://api.yelp.com/v3/businesses/search,1653,4.0,$$,178 Ludlow St,,,New York,NY,10002,+16466570233
24877,NcHnZbq06y9llCt6ayf_Jw,Sweet Chick,https://s3-media3.fl.yelpcdn.com/bphoto/YXQSUp...,https://api.yelp.com/v3/businesses/search,333,4.0,$$,46-42 Vernon Blvd,,None,Long Island City,NY,11101,+17184334064
